# Data Specs

No valid data value = 255

The reference is the centre of the pixel. It means that the longitude of the upper left corner of the pixel is (pixel_longitude – angular_resolution/2.)

# Import modules

In [ ]:
from irrigation_detection import download_NDVI_max
from irrigation_detection import unzip_ndvi
from irrigation_detection import open_NDVI
from irrigation_detection import subset_ds
import numpy as np
from tqdm import tqdm
import pickle
import xarray as xr
import hvplot.xarray

# Get data
https://land.copernicus.eu/global/products/ndvi

In [ ]:
# Download data from ftp server after purchase it from copernicus.
download_NDVI_max('till90', "201184", 'M0118447','D://ndvi_download/zip_archives/')

In [ ]:
# Extract archives.
unzip_ndvi('D://ndvi_download/zip_archives/','D://ndvi_download/data/')

# Data import and subsetting

In [ ]:
# Read extraced ndvi files with xarray.
NDVI_ds = open_NDVI('D://ndvi_download/data')

In [ ]:
# Subset NDVI raster.
NDVI_ds_subset = subset_ds(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\oberrheingraben_shp\oberrheingraben-polygon.shp',NDVI_ds)

# Data preprocessing

## digital to physical values 
| LAYER NAME |                                                                                                       DESCRIPTION                                                                                                      | PHYSICAL UNIT / CLASSES | PHYSICAL MIN | PHYSICAL MAX | DIGITAL MAX | SCALING | OFFSET |
|:----------:|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------:|:-----------------------:|:------------:|:------------:|:-----------:|:-------:|:------:|
| NDVI       | Normalized Difference Vegetation Index                                                                                                                                                                                 | -                       | -0.08        | 0.92         | 250         | 1/250   | -0.08  |
| NDVI_unc   | Uncertainty associated to NDVI                                                                                                                                                                                         | -                       | 0            | 1            | 1000        | 1/1000  | 0      |
| NOBS       | Number of observations used in the BRDF inversion                                                                                                                                                                      | -                       | 0            | 32           | 32          | 1       | 0      |
| QFLAG      | Quality flag                                                                                                                                                                                                           | N/A                     | N/A          | N/A          | 16          | 1       | 0      |
| TIME_GRID  |   Time between the median date of observations used to compute NDVI and the reference time given in the file name.   It is the most representative date of the observations and recommended for time series analysis.  | Minutes                 | -8640        | 15840        | 15840       | 1       | 0      |

The physical values (PV) are derived from the digital number (DN) using the relation: PV = Scaling * DN + Offset  


In [ ]:
# Convert to physical numbers and mask out invalid values.
NDVI_ds_subset =  1/250 * NDVI_ds_subset.NDVI.where(NDVI_ds_subset.NDVI <=250, drop=True) - 0.08

In [ ]:
# Interpolate and Fill nan values
NDVI_ds_subset = NDVI_ds_subset.interpolate_na(dim='time').bfill(dim='time').ffill(dim='time')

In [ ]:
# Export as netcdf to disk
NDVI_ds_subset.to_netcdf(r'C:\Users\USER\Desktop\Master_Irrigation\03_GIS\datasets\NDVI_10Dmax_1km.nc')

# Determine irrigation Period (IP)

## IP_ndvi

In [ ]:
#filter for months between ints
def is_may_september(month):
    return (month >= 5) & (month <= 8)

In [ ]:
test =  1/250 * NDVI_ds_subset.NDVI.where(NDVI_ds_subset.NDVI <=250, drop=True) - 0.08

In [ ]:
test.hvplot.line(x='time', ylim=[0,1])

In [ ]:
IP_ndvi = list()
#iterate over groupby object 
#Rolling mean for noise reduction
ndvi = NDVI_ds_subset.NDVI.rolling(time=35).mean()
#stack latitude and longitude to prepare groupby gridcell
ndvi = NDVI_ds_subset.NDVI.stack(gridcell=["lat", "lon"])
#group into single cells
for i, (gridcell, grouped_array) in zip(tqdm(range(len(ndvi.groupby('gridcell')))), ndvi.groupby('gridcell')):
    #group single gridcells into years
    for year, ga in grouped_array.groupby('time.year'):
        try:
            #valid values between 0 and 250, drop other e.g 255
            masked_array = ga.where(ga <= 250, drop=True)
            masked_array = 1/250 * masked_array - 0.08
            #indexing values between april and october
            array_peak = masked_array.sel(time=is_may_september(masked_array['time.month']))
            #find peak values
            array_peak = array_peak.idxmax(dim='time', skipna=True)
            #date 4 months before peak 
            time_range_min = array_peak - np.timedelta64(120, 'D')
            #date 4 months after peak
            time_range_max = array_peak + np.timedelta64(120, 'D')
            #values between min and peak
            array_sos_peak = masked_array.sel(time=slice(time_range_min.data[0],array_peak.data[0]))
            #values between peak and min 
            array_peak_eos = masked_array.sel(time=slice(array_peak.data[0],time_range_max.data[0]))
            #ndvi values 20 percent higher than min ndvi
            ndvi_min_20psos = array_sos_peak.min() * 1.2
            ndvi_min_20peos = array_peak_eos.min() * 1.2
            #find eos and sos 
            try:
                #start of season is when ndvi is at least 20 percent higher than min
                sos = array_sos_peak.where(array_sos_peak >= ndvi_min_20psos, drop=True).isel(time=1)
                eos = array_peak_eos.where(array_peak_eos >= ndvi_min_20peos, drop=True).isel(time=-1)
                #get days between sos and eos
                days = eos.time.data - sos.time.data
                #add values to list if criteria is fulfill (difference between peak and min not smaller than 0.2 and more than 75 days irrigation period)
                if array_peak.data - sos.data > 0.2 and int(days) / 8.64e+13 > 75:
                    IP_ndvi.append((gridcell,year,sos.time.data,eos.time.data))
                else:
                    IP_ndvi.append((gridcell,year,np.nan,np.nan))
            except:
                IP_ndvi.append((gridcell,year,np.nan,np.nan))
        except:
            IP_ndvi.append((gridcell,year,np.nan,np.nan))

#save list to pickle
with open('IP_ndvi.pkl', 'wb') as file:
    pickle.dump(IP_ndvi, file)

In [ ]:
#create mask coordinate to mask out nans
ndvi.coords['mask'] = (('lat', 'lon'), np.isnan(ndvi.NDVI.isel(time=1).data))

In [ ]:
ds.coords['mask'] = (('latitude', 'longitude'), mask_array)
https://hvplot.holoviz.org/user_guide/Geographic_Data.html

In [ ]:
def irrigation_period_ndvi(ds):
    """
    Arguments: ds
    """
    
    import xarray as xr
    
    ndvi = ds.groupby('time', )

In [ ]:
import hvplot.xarray

In [ ]:
hvplot.save(ndvi.hvplot.quadmesh(crs='crs',frame_height=540, project=True, geo=True, rasterize=True, dynamic=False), "ndvi.html")